# DX 704 Week 5 Project

This week's project will test your understanding of this week's concepts by asking you to simulate various algorithms by hand.
You will apply search, minimax and dynamic programming concepts to solve a variety of small planning problems.

The full project description, a template notebook and supporting materials are available on GitHub: [Project 5 Materials](https://github.com/bu-cds-dx704/dx704-project-05).

## Example Code

This week's assignment does not involve any coding.

## Part 1: Searching vs Games

Consider the state space illustrated below.
Each terminal state state is marked with a reward for reaching that state.
Each non-terminal state has two possible actions represented by the two outgoing arrows to later (lower) states.
The only rewards are for reaching the terminal states, there are no diminishing returns (i.e. $\gamma=1$), and there is no randomness so actions may be freely chosen.

![](https://github.com/bu-cds-dx704/dx704-project-05/blob/main/part1.png?raw=true)

Solve for the value of each non-terminal state according to the following three scenarios.

1. Search: There is one agent that picks all actions with the goal of maximizing the final reward.
2. Minimax: There are two agents P1 and P2. P1 controls the actions for the 1st and 3rd rows (i.e. the states marked A and D-G) while P2 controls the actions for the 2nd and 4th rows (i.e. the states B-C and H-J). P1 seeks to maximize the final reward, and P2 seeks to minimize the final reward.
3. Maximin: P1 and P2 control the same states as before, but P1 seeks to minimize the final reward, and P2 seeks to maximize the final reward.

Save your results in a file "values-1.tsv" with the column state with label A-J and columns search_value, minimax_value, and maximin_value that respectively correspond to the three scenarios.

Hint: Print out the image above and compute the values by hand in a bottom up fashion.

In [27]:
# === Part 1: bottom-up evaluation over part1.dot ===
# Output columns EXACTLY: state, search, minimax, maximin

from pathlib import Path
from collections import defaultdict, deque
from functools import lru_cache
import re, csv

DOT_PATH = Path("part1.dot")

edge_re  = re.compile(r'(\w+)\s*->\s*(\w+)\s*;')
node_decl_re = re.compile(r'(\w+)\s*\[(.*?)\]', re.DOTALL)
label_re = re.compile(r'label\s*=\s*(?:"([^"]+)"|<([^>]+)>|([^\s,\]]+))', re.DOTALL)

raw = DOT_PATH.read_text()

# edges
adj = defaultdict(list); nodes = set()
for a,b in edge_re.findall(raw):
    adj[a].append(b); nodes.add(a); nodes.add(b)

# labels
id_to_label = {}
for node_id, attrs in node_decl_re.findall(raw):
    m = label_re.search(attrs)
    if m:
        lbl = (m.group(1) or m.group(2) or m.group(3)).strip()
        id_to_label[node_id] = lbl
    nodes.add(node_id)

# normalize labels to bare state names (e.g., "S: start" -> "S", "C: +100" -> "C")
def norm(lbl):
    m = re.match(r'([A-Za-z]+)', lbl.strip())
    return m.group(1) if m else lbl.strip()

label_to_id = {norm(v): k for k,v in id_to_label.items()}
root = label_to_id.get("A")
if root is None:
    indeg = {u:0 for u in nodes}
    for u, vs in adj.items():
        for v in vs:
            indeg[v] = indeg.get(v, 0) + 1
            indeg.setdefault(u, 0)
    zero_in = [u for u,d in indeg.items() if d == 0]
    root = zero_in[0] if zero_in else next(iter(nodes))

# depths
depth = {root: 0}
dq = deque([root])
while dq:
    u = dq.popleft()
    for v in adj.get(u, []):
        if v not in depth:
            depth[v] = depth[u] + 1
            dq.append(v)

def is_num(lbl): 
    return re.fullmatch(r'[+-]?\d+', lbl.strip()) is not None

# leaf values (must be numeric)
leaf_values = {}
for n in nodes:
    if len(adj.get(n, [])) == 0:
        lbl = id_to_label.get(n, "")
        if not is_num(lbl):
            raise RuntimeError(f"Leaf {n} lacks numeric label: '{lbl}'")
        leaf_values[n] = int(lbl)

@lru_cache(None)
def kids(n): return tuple(adj.get(n, []))

@lru_cache(None)
def val_search(n):
    if n in leaf_values: return leaf_values[n]
    return max(val_search(c) for c in kids(n))

@lru_cache(None)
def val_minimax(n):
    if n in leaf_values: return leaf_values[n]
    d = depth[n]
    vals = [val_minimax(c) for c in kids(n)]
    return max(vals) if d % 2 == 0 else min(vals)

@lru_cache(None)
def val_maximin(n):
    if n in leaf_values: return leaf_values[n]
    d = depth[n]
    vals = [val_maximin(c) for c in kids(n)]
    return min(vals) if d % 2 == 0 else max(vals)

# Build rows in terms of CLEAN state names
rows = []
for nid in nodes:
    lbl_full = id_to_label.get(nid, nid)
    state = norm(lbl_full)
    rows.append((state, val_search(nid), val_minimax(nid), val_maximin(nid)))

# De-duplicate (if multiple IDs map to same state label)
agg = {}
for st, s, mm, mx in rows:
    # prefer root-consistent or any (all should agree)
    agg[st] = (s, mm, mx)

# Stable order: alphabetic by state
out = [(st,)+agg[st] for st in sorted(agg.keys())]

with open("values-1.tsv", "w", newline="") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["state","search","minimax","maximin"])
    for row in out:
        w.writerow(row)

print("Wrote values-1.tsv with", len(out), "states")



Wrote values-1.tsv with 18 states


Submit the file "values-1.tsv" in Gradescope.

## Part 2: Picking Up Sticks

The state space illustrated below corresponds to a variation of the game [Nim](https://en.wikipedia.org/wiki/Nim).
States labeled with a prefix of "p1_" correspond to states where player P1 chooses the action while states labeled with a prefix of "p2_" correspond to states where player P2 chooses the action.
The number in the suffix is the number of "sticks" remaining.
The players take turns choosing actions, and each action corresponds to removing one or two sticks.
When there are no more sticks, the player who would have picked an action loses.


![](https://github.com/bu-cds-dx704/dx704-project-05/blob/main/part2.png?raw=true)

For example, from the state labeled "p1_1", there is one stick left, player P1 removes the last stick, and player P2 loses.
The loss for P2 is represented by a final reward of +1.
A loss for P1 is represented by a final reward of -1.
Player P1 tries to maximize the final reward, and player P2 tries to minimize the final reward.

Solve for the value of each of the non-terminal states.
Save the results in a file "values-2.tsv" with columns state and value.

In [28]:
from pathlib import Path
from collections import defaultdict
from functools import lru_cache
import re, csv

DOT_PATH = Path("part2.dot")  

def parse_dot_edges(dot_path: Path):
    text = dot_path.read_text()
    edge_pat = re.compile(r'(\w+)\s*->\s*(\w+)\s*;')
    adj = defaultdict(list); nodes = set()
    for a,b in edge_pat.findall(text):
        adj[a].append(b); nodes.add(a); nodes.add(b)
    return dict(adj), nodes

adj, nodes = parse_dot_edges(DOT_PATH)

def is_state(s: str):
    return re.fullmatch(r'p[12]_\d+', s) is not None

states = sorted([s for s in nodes if is_state(s)],
                key=lambda s: (int(s.split('_')[1]), s.split('_')[0]))

TERMINAL = {"p1_0": -1, "p2_0": +1}

@lru_cache(None)
def negamax(s: str) -> int:
    if s in TERMINAL:
        return TERMINAL[s]
    succ = adj.get(s, [])
    if not succ:
        return 0  # defensive
    # current-player value = max over successors of -value(next)
    return max(-negamax(t) for t in succ)

with open("values-2.tsv", "w", newline="") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["state", "value"])
    for s in states:
        w.writerow([s, negamax(s)])

print("Wrote values-2.tsv with", len(states), "rows")





Wrote values-2.tsv with 12 rows


Submit the file "values-2.tsv" in Gradescope.

## Part 3: Searching a Maze

Consider the following maze.

![](https://github.com/bu-cds-dx704/dx704-project-05/blob/main/part3.png?raw=true)

State C is a terminal state giving reward +100.
The remaining states have a reward of -1 when they are reached.
So moving to state F has a value of +99 do to the reward of -1 at state F and the optimal action of moving to state C for the reward of +100 afterwards.

Compute the values for states A-J and S and save them in a file "values-3.tsv" with columns state and value.

In [29]:
# === Part 3: V*(s) = 100 - shortest_steps_to_C (γ=1, r=-1 per nonterminal, r(C)=100) ===
# Output columns EXACTLY: state, value   (INCLUDE C with value 100)

from pathlib import Path
from collections import defaultdict, deque
import re, csv, math

DOT_PATH = Path("part3.dot")  # MUST be the MDP graph (the one with S and C)

edge_re       = re.compile(r'(\w+)\s*->\s*(\w+)\s*;')
node_decl_re  = re.compile(r'(\w+)\s*\[(.*?)\]', re.DOTALL)
label_re      = re.compile(r'label\s*=\s*(?:"([^"]+)"|<([^>]+)>|([^\s,\]]+))', re.DOTALL)

raw = DOT_PATH.read_text()

adj = defaultdict(list); nodes = set()
for a,b in edge_re.findall(raw):
    adj[a].append(b); nodes.add(a); nodes.add(b)

id_to_label = {}
for node_id, attrs in node_decl_re.findall(raw):
    m = label_re.search(attrs)
    if m:
        id_to_label[node_id] = (m.group(1) or m.group(2) or m.group(3)).strip()
    nodes.add(node_id)

def norm(lbl):
    m = re.match(r'([A-Za-z]+)', lbl.strip())
    return m.group(1) if m else lbl.strip()

# find C (label has +100 or begins with C)
def is_C(lbl):
    t = lbl.strip()
    return ('+100' in t) or t.startswith('C')

C_nodes = [nid for nid,lbl in id_to_label.items() if is_C(lbl)]
if not C_nodes:
    raise RuntimeError("Terminal C not found (look for label containing '+100' or starting with 'C').")
C = C_nodes[0]

# reverse BFS from C to get shortest steps
rev = defaultdict(list)
for u, vs in adj.items():
    for v in vs:
        rev[v].append(u)

dist = {n: math.inf for n in nodes}
dq = deque([C]); dist[C] = 0
while dq:
    x = dq.popleft()
    for p in rev.get(x, []):
        if dist[p] == math.inf:
            dist[p] = dist[x] + 1
            dq.append(p)

# write TSV (include C=100). Use normalized labels like A,B,C,S (strip ": ...")
rows = []
for nid in nodes:
    lbl = norm(id_to_label.get(nid, nid))
    if dist[nid] == math.inf:
        # unreachable → -inf under γ=1, step cost −1
        val = "-inf"
    else:
        val = 100 - dist[nid] if nid != C else 100
    rows.append((lbl, val))

# de-duplicate by state label
best = {}
for s,v in rows:
    best[s] = v

out = sorted(best.items(), key=lambda t: t[0])
with open("values-3.tsv", "w", newline="") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["state","value"])
    for s,v in out:
        w.writerow([s, v])

print("Wrote values-3.tsv with", len(out), "states")



Wrote values-3.tsv with 11 states


Submit "values-3.tsv" in Gradescope.

## Part 4: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

None

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

None

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

here is the conversation i had with chatgpt helping me go over my work

https://chatgpt.com/share/68e1d9c3-46d4-800d-9203-4fd1db4f6195

In [30]:
with open("acknowledgments.txt", "w") as f:
    f.write("Acknowledgments: I discussed general approach with ChatGPT for debugging formatting and file outputs. Here is the link: https://chatgpt.com/share/68e1d9c3-46d4-800d-9203-4fd1db4f6195 \n")
print("Wrote acknowledgments.txt")


Wrote acknowledgments.txt
